In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline 


In [25]:
df = sns.load_dataset('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [26]:
df.describe()

,total_bill,tip,size
count,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672
std,8.902412,1.383638,0.951100
min,3.070000,1.000000,1.000000
25%,13.347500,2.000000,2.000000
50%,17.795000,2.900000,2.000000
75%,24.127500,3.562500,3.000000
max,50.810000,10.000000,6.000000


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   total_bill  244 non-null    float64 
 1   tip         244 non-null    float64 
 2   sex         244 non-null    category
 3   smoker      244 non-null    category
 4   day         244 non-null    category
 5   time        244 non-null    category
 6   size        244 non-null    int64   
dtypes: category(4), float64(2), int64(1)
memory usage: 7.4 KB


In [28]:
df['time'].unique()

['Dinner', 'Lunch']
Categories (2, object): ['Lunch', 'Dinner']

In [29]:
df.isnull().sum()

total_bill    0
tip           0
sex           0
smoker        0
day           0
time          0
size          0
dtype: int64

In [30]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['time']=encoder.fit_transform(df['time'])

In [31]:
df['time'].unique()

array([0, 1])

In [32]:
df.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,0,2
1,10.34,1.66,Male,No,Sun,0,3
2,21.01,3.50,Male,No,Sun,0,3
3,23.68,3.31,Male,No,Sun,0,2
4,24.59,3.61,Female,No,Sun,0,4


In [33]:
X = df.drop(labels = ['time'],axis=1)
y = df['time']

In [34]:
X.head()

,total_bill,tip,sex,smoker,day,size
0,16.99,1.01,Female,No,Sun,2
1,10.34,1.66,Male,No,Sun,3
2,21.01,3.50,Male,No,Sun,3
3,23.68,3.31,Male,No,Sun,2
4,24.59,3.61,Female,No,Sun,4


In [35]:
X['day'].value_counts()

Sat     87
Sun     76
Thur    62
Fri     19
Name: day, dtype: int64

In [106]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20,random_state=42)

In [40]:
from sklearn.pipeline import Pipeline # to automate task 
from sklearn.impute import SimpleImputer # Handle missing values
from sklearn.preprocessing import StandardScaler #Feature Scaling
from sklearn.preprocessing import OneHotEncoder ## categorical to numerical
from sklearn.compose import ColumnTransformer

In [41]:
categorical_cols = ['sex','smoker','day']
numerical_cols = ['total_bill','tip','size']

## Feature Engineering Automation

## Numerical Pipeline

In [44]:
num_pipeline = Pipeline(
    steps = [ 
        ('imputer',SimpleImputer(strategy="median")), # to handle missing values using median 
        ('scaler',StandardScaler())
    ]
)

## Categorical Pipeline 

In [45]:
categ_pipeline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy="most_frequent")), # handle missing values using mode 
        ('onehotencoder',OneHotEncoder())   # convert categorical columns to numerical
    ]
)

## Column Transformer

In [46]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',categ_pipeline,categorical_cols)
]
)

In [107]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [108]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## Model Training  Automation

In [65]:
# store different ML Algorithms that we need to apply
models={
    'Random Forest':RandomForestClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Decision Tree':DecisionTreeClassifier(),
    'SVC' : SVC()
}

In [51]:
from sklearn.metrics import accuracy_score

In [55]:
def evaluate_model(X_train,X_test,y_train,y_test,models):
    report={}
    for i in range(len(models)):
        model=list(models.values())[i]
        #train model
        model.fit(X_train,y_train)
        #predicting test data
        y_test_pred=model.predict(X_test)
        #get accuracy score
        test_score = accuracy_score(y_test,y_test_pred)
        #inserting in report dictionary
        report[list(models.keys())[i]]=test_score
    return report 


## Accuracy score of diferent ML Algorithms

In [64]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest': 0.9795918367346939,
 'Logistic Regression': 1.0,
 'Decision Tree': 0.9795918367346939,
 'SVC': 0.9795918367346939}

## Hyperparameter Tuning 

In [66]:
classfier=RandomForestClassifier()

In [67]:
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300],
               'criterion':['gini','entropy']
              }

In [68]:
from sklearn.model_selection import RandomizedSearchCV

In [69]:
cv=RandomizedSearchCV(classfier,param_distributions=params,scoring='accuracy',cv=5,verbose=3)
cv.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 2/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 3/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.974 total time=   0.2s
[CV 4/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 5/5] END criterion=gini, max_depth=5, n_estimators=100;, score=0.923 total time=   0.2s
[CV 1/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.974 total time=   0.4s
[CV 2/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.923 total time=   0.4s
[CV 3/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=1.000 total time=   0.4s
[CV 4/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.949 total time=   0.4s
[CV 5/5] END criterion=entropy, max_depth=None, n_estimators=200;, score=0.923 total ti

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(),
                   param_distributions={'criterion': ['gini', 'entropy'],
                                        'max_depth': [3, 5, 10, None],
                                        'n_estimators': [100, 200, 300]},
                   scoring='accuracy', verbose=3)

In [70]:
cv.best_params_

{'n_estimators': 200, 'max_depth': None, 'criterion': 'entropy'}

In [78]:
models={
    'Random Forest Classifier':RandomForestClassifier(n_estimators=200,max_depth=None,criterion='entropy'),
}

In [79]:
evaluate_model(X_train,X_test,y_train,y_test,models)

{'Random Forest Classifier': 0.9795918367346939}

## Random Forest Regressor 

In [76]:
from sklearn.ensemble import RandomForestRegressor

In [77]:
classifier = RandomForestRegressor()

In [86]:
params={'max_depth':[3,5,10,None],
              'n_estimators':[100,200,300,400,500],
               'criterion':['squared_error','absolute_error','friedman_mse','poisson'],
        'max_features':['sqrt','log2',None]
        }

In [89]:
clf = RandomizedSearchCV(classifier,param_distributions=params,scoring='neg_mean_squared_error',cv=5,verbose=3)

In [90]:
clf.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV 1/5] END criterion=poisson, max_depth=5, max_features=log2, n_estimators=300;, score=-0.053 total time=   0.5s
[CV 2/5] END criterion=poisson, max_depth=5, max_features=log2, n_estimators=300;, score=-0.093 total time=   0.5s
[CV 3/5] END criterion=poisson, max_depth=5, max_features=log2, n_estimators=300;, score=-0.060 total time=   0.5s
[CV 4/5] END criterion=poisson, max_depth=5, max_features=log2, n_estimators=300;, score=-0.126 total time=   0.5s
[CV 5/5] END criterion=poisson, max_depth=5, max_features=log2, n_estimators=300;, score=-0.071 total time=   0.5s
[CV 1/5] END criterion=squared_error, max_depth=5, max_features=log2, n_estimators=100;, score=-0.013 total time=   0.2s
[CV 2/5] END criterion=squared_error, max_depth=5, max_features=log2, n_estimators=100;, score=-0.050 total time=   0.2s
[CV 3/5] END criterion=squared_error, max_depth=5, max_features=log2, n_estimators=100;, score=-0.017 total time=   0.2s
[

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(),
                   param_distributions={'criterion': ['squared_error',
                                                      'absolute_error',
                                                      'friedman_mse',
                                                      'poisson'],
                                        'max_depth': [3, 5, 10, None],
                                        'max_features': ['sqrt', 'log2', None],
                                        'n_estimators': [100, 200, 300, 400,
                                                         500]},
                   scoring='neg_mean_squared_error', verbose=3)

In [95]:
clf.best_params_

{'n_estimators': 100,
 'max_features': 'log2',
 'max_depth': 5,
 'criterion': 'squared_error'}

In [96]:
classifier = RandomForestRegressor(n_estimators=100,max_features='log2',max_depth=5,criterion='squared_error')

In [97]:
classifier.fit(X_train,y_train)

RandomForestRegressor(max_depth=5, max_features='log2')

In [99]:
y_pred=classifier.predict(X_test)

In [101]:
from sklearn.metrics import r2_score
r2_score(y_pred,y_test)

0.9347134246397477